In [1]:
import numpy as np
import sys

def gram_schmidt(vectors, normalize=True):
    n = vectors.shape[1]  # Number of vectors
    m = vectors.shape[0]  # Vector dimension
    
    # Create a copy to store the orthogonalized vectors
    orthogonal_vectors = np.zeros((m, n))
    
    # Process each vector
    for i in range(n):
        # Start with the original vector
        orthogonal_vectors[:, i] = vectors[:, i]
        
        # Subtract projections onto previous orthogonal vectors
        for j in range(i):
            # Calculate projection
            projection = np.dot(vectors[:, i], orthogonal_vectors[:, j])
            if normalize:
                projection /= np.dot(orthogonal_vectors[:, j], orthogonal_vectors[:, j])
            
            # Subtract projection
            orthogonal_vectors[:, i] -= projection * orthogonal_vectors[:, j]
            
            # Print intermediate step
            print(f"\nStep {i+1}.{j+1}: Subtracting projection onto vector {j+1}")
            print(f"Projection coefficient: {projection:.6f}")
            print(f"Current result: {orthogonal_vectors[:, i]}")
        
        # Normalize if requested
        if normalize:
            norm = np.linalg.norm(orthogonal_vectors[:, i])
            if norm > 1e-10:  # Avoid division by zero
                orthogonal_vectors[:, i] /= norm
                print(f"\nStep {i+1}.{i+1}: Normalizing vector {i+1}")
                print(f"Norm: {norm:.6f}")
                print(f"Normalized vector: {orthogonal_vectors[:, i]}")
            else:
                print(f"\nWarning: Vector {i+1} has zero norm after orthogonalization!")
                print("This suggests linear dependence in the input vectors.")
    
    return orthogonal_vectors

def get_integer_input(prompt, min_value=1):
    """Get an integer input from the user with validation."""
    while True:
        try:
            value = int(input(prompt))
            if value < min_value:
                print(f"Please enter a value >= {min_value}")
            else:
                return value
        except ValueError:
            print("Please enter a valid integer")

def get_vector_input(dim, idx):
    """Get a vector input from the user."""
    while True:
        try:
            print(f"\nEnter the elements of vector {idx+1} (dimension {dim}), separated by commas:")
            elements = input().strip().split(',')
            
            if len(elements) != dim:
                print(f"Error: You entered {len(elements)} elements, but dimension is {dim}")
                continue
            
            vector = np.array([float(e.strip()) for e in elements])
            return vector
        except ValueError:
            print("Error: Please enter numeric values separated by commas")

def check_linear_independence(vectors):
    """Check if the input vectors are linearly independent."""
    # For a set of vectors to be linearly independent, the matrix must have full rank
    matrix = vectors.copy()
    rank = np.linalg.matrix_rank(matrix)
    
    if rank < matrix.shape[1]:
        print("\nWarning: The input vectors are linearly dependent!")
        print(f"Rank of the matrix: {rank}, Number of vectors: {matrix.shape[1]}")
        return False
    return True

def main():
    print("==== Gram-Schmidt Orthogonalization ====")
    
    # Get dimensions from user
    dim = get_integer_input("Enter the dimension of the vectors: ")
    num_vectors = get_integer_input("Enter the number of vectors: ")
    
    # Ensure we're not asking for too many orthogonal vectors
    if num_vectors > dim:
        print(f"Warning: You can have at most {dim} orthogonal vectors in {dim}-dimensional space.")
        print(f"Setting the number of vectors to {dim}.")
        num_vectors = dim
    
    # Get the vectors from user
    vectors = np.zeros((dim, num_vectors))
    for i in range(num_vectors):
        vectors[:, i] = get_vector_input(dim, i)
    
    print("\nInput vectors (as columns):")
    print(vectors)
    
    # Check linear independence
    if not check_linear_independence(vectors):
        proceed = input("\nThe vectors may be linearly dependent. Proceed anyway? (y/n): ")
        if proceed.lower() != 'y':
            print("Exiting program.")
            sys.exit(0)
    
    # Ask whether to normalize
    normalize = input("\nCreate orthonormal basis (y) or just orthogonal (n)? ").lower() == 'y'
    
    print("\nPerforming Gram-Schmidt orthogonalization...")
    orthogonal_vectors = gram_schmidt(vectors, normalize)
    
    print("\nResult:")
    print("Orthogonal" if not normalize else "Orthonormal", "vectors (as columns):")
    print(orthogonal_vectors)
    
    # Verify orthogonality by computing dot products
    print("\nVerification - dot products between result vectors:")
    for i in range(num_vectors):
        for j in range(i+1, num_vectors):
            dot_product = np.dot(orthogonal_vectors[:, i], orthogonal_vectors[:, j])
            print(f"Dot product between vectors {i+1} and {j+1}: {dot_product:.10f}")
    
    # Verify normalization if requested
    if normalize:
        print("\nVerification - norms of result vectors:")
        for i in range(num_vectors):
            norm = np.linalg.norm(orthogonal_vectors[:, i])
            print(f"Norm of vector {i+1}: {norm:.10f}")

if __name__ == "__main__":
    main()

==== Gram-Schmidt Orthogonalization ====

Enter the elements of vector 1 (dimension 3), separated by commas:

Enter the elements of vector 2 (dimension 3), separated by commas:

Input vectors (as columns):
[[2. 1.]
 [2. 4.]
 [3. 5.]]

Performing Gram-Schmidt orthogonalization...

Step 1.1: Normalizing vector 1
Norm: 4.123106
Normalized vector: [0.48507125 0.48507125 0.72760688]

Step 2.1: Subtracting projection onto vector 1
Projection coefficient: 6.063391
Current result: [-1.94117647  1.05882353  0.58823529]

Step 2.2: Normalizing vector 2
Norm: 2.288077
Normalized vector: [-0.84838792  0.46275705  0.25708725]

Result:
Orthonormal vectors (as columns):
[[ 0.48507125 -0.84838792]
 [ 0.48507125  0.46275705]
 [ 0.72760688  0.25708725]]

Verification - dot products between result vectors:
Dot product between vectors 1 and 2: 0.0000000000

Verification - norms of result vectors:
Norm of vector 1: 1.0000000000
Norm of vector 2: 1.0000000000
